In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import math
from scipy.stats import f_oneway

def readfile(filename):
    # Get the data from the file
    file = os.path.join(os.getcwd(), 'output', 'files', str(filename))
    df = pd.read_csv(file, index_col=False)

    # Drop unnecessary columns, get exit and entry frame.
    df = df.drop(['Cell kind', 'Coordinates'], axis=1)
    df['exit'] = df.loc[df['Event'] == 'exit', 'Frame'].astype(float)
    df['entry'] = df.loc[df['Event'] == 'entry', 'Frame'].astype(float)
    df = df.drop(['Event', 'Frame'], axis=1)

    # Get run number
    df['Run'] = np.nan
    run = 1
    for i, row in df.iterrows():
        if row[0] == 'ID':
            run += 1
        else:
            df.at[i, 'Run'] = run

    # Group runs and remove info of cells which haven't finished (last 20)
    df = df[df['ID'] != 'ID']
    dfs = [x for _, x in df.groupby('Run')]
    for i in range(len(dfs)):
        dfs[i] = dfs[i].groupby(['ID'], as_index=False).first()
    df = pd.concat(dfs)
    df = df.dropna(subset=['exit'])

    # Remove first 20 cells due to uninhibited movement
    for run in df['Run'].unique():
        ids_to_remove = df[df['Run'] == run]['ID'].unique()[:20]
        df = df[~((df['Run'] == run) & (df['ID'].isin(ids_to_remove)))]

    # Casting dtypes
    df['Frames'] = df['exit'].astype(float) - df['entry'].astype(float)
    df['Run'] = df['Run'].astype(int)
    runs = df['Run'].unique()

    # Creating final dataframe
    summary_df = pd.DataFrame(columns=['Run', 'Mean Frames', 'Std Frames', 'Min Frames', 'Max Frames',
                                       'Median Frames', 'Q1 Frames', 'Q3 Frames'])

    for run in df['Run'].unique():
        df_run = df[df['Run'] == run]
        mean_frames = df_run['Frames'].mean()
        std_frames = df_run['Frames'].std()
        min_frames = df_run['Frames'].min()
        max_frames = df_run['Frames'].max()
        median_frames = df_run['Frames'].median()
        q1_frames = df_run['Frames'].quantile(0.25)
        q3_frames = df_run['Frames'].quantile(0.75)

        row = pd.DataFrame({'Run': [run], 'Mean Frames': [mean_frames], 'Std Frames': [std_frames],
                            'Min Frames': [min_frames], 'Max Frames': [max_frames],
                            'Median Frames': [median_frames], 'Q1 Frames': [q1_frames], 'Q3 Frames': [q3_frames]})

        summary_df = pd.concat([summary_df, row], ignore_index=True)

    # Creating final averaging row
    average_all_runs = summary_df.mean().to_frame().T
    average_all_runs['Run'] = 'Average'
    summary_df = pd.concat([summary_df, average_all_runs], ignore_index=True)

    return summary_df, df


In [2]:
filenames = ['lambda_act_100_setting_1.csv',
            'lambda_act_100_setting_2.csv',
            'lambda_act_100_setting_3.csv',
            'lambda_act_100_setting_4.csv',
            'lambda_act_200_setting_1.csv',
            'lambda_act_200_setting_2.csv',
            'lambda_act_200_setting_3.csv',
            'lambda_act_200_setting_4.csv',
            'lambda_act_300_setting_1.csv',
            'lambda_act_300_setting_2.csv',
            'lambda_act_300_setting_3.csv',
            'lambda_act_300_setting_4.csv']

all_data = {}
for file in filenames:
    all_data[file] = readfile(file)

C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(file, index_col=False)
C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(file, index_col=False)
C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(file, index_col=False)
C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(file, index_col=False)
C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: Parse

In [3]:
import matplotlib.pyplot as plt

# Maak een plot voor het gemiddeld aantal frames per run
plt.figure(figsize=(10,6))
plt.plot(summary_df['Run'][:-1], summary_df['Mean Frames'][:-1])  # we sluiten het laatste gemiddelde uit
plt.xlabel('Run')
plt.ylabel('Gemiddeld aantal Frames')
plt.title('Gemiddeld aantal frames per run')
plt.grid(True)
plt.show()

# Maak een histogram van de gemiddelde frames
plt.figure(figsize=(10,6))
plt.hist(full_df['Frames'], bins=20, edgecolor='black')
plt.xlabel('Aantal Frames')
plt.ylabel('Frequentie')
plt.title('Histogram van het aantal Frames')
plt.grid(True)
plt.show()

# Boxplot van Frames per Run
full_df.boxplot(column='Frames', by='Run', figsize=(10,6))
plt.title('Boxplot van Frames per Run')
plt.suptitle('')  # Verwijder de automatische 'Boxplot grouped by Run' titel
plt.show()

# Scatterplot van 'exit' en 'entry'
plt.figure(figsize=(10,6))
plt.scatter(full_df['exit'], full_df['entry'])
plt.xlabel('Exit Frame')
plt.ylabel('Entry Frame')
plt.title('Scatterplot van Exit vs Entry Frames')
plt.grid(True)
plt.show()


NameError: name 'summary_df' is not defined

<Figure size 1000x600 with 0 Axes>

In [4]:
def z_test(run1, run2):
    data1 = all_data[run1][0]
    data2 = all_data[run2][0]
    avg1 = data1.loc[100]['Mean Frames']
    avg2 = data2.loc[100]['Mean Frames']
    std1 = data1.loc[100]['Std Frames']
    std2 = data2.loc[100]['Std Frames']
    return (avg1 - avg2) / (math.sqrt(std1 ** 2 + std2 ** 2))

keys = list(all_data.keys())
z_test(keys[0], keys[1])

-0.2577266237933793

# $\lambda$Act Analysis


In [98]:
for x in keys:
    print(f'Name is {x} with mean frames of: {all_data[x][0].loc[100]["Mean Frames"]}')
    print(f'Name is {x} with stdev of: {all_data[x][0].loc[100]["Std Frames"]}')

Name is lambda_act_100_setting_1.csv with mean frames of: 1569.9441982310937
Name is lambda_act_100_setting_1.csv with stdev of: 199.73139183808425
Name is lambda_act_100_setting_2.csv with mean frames of: 1635.4987571476431
Name is lambda_act_100_setting_2.csv with stdev of: 157.4954952120743
Name is lambda_act_100_setting_3.csv with mean frames of: 1635.2814830444026
Name is lambda_act_100_setting_3.csv with stdev of: 146.55830756748105
Name is lambda_act_100_setting_4.csv with mean frames of: 1583.9637180551167
Name is lambda_act_100_setting_4.csv with stdev of: 185.74911828719877
Name is lambda_act_200_setting_1.csv with mean frames of: 1241.0281967213114
Name is lambda_act_200_setting_1.csv with stdev of: 173.37068741071016
Name is lambda_act_200_setting_2.csv with mean frames of: 1278.223655737705
Name is lambda_act_200_setting_2.csv with stdev of: 160.06436784313655
Name is lambda_act_200_setting_3.csv with mean frames of: 1285.1547213114757
Name is lambda_act_200_setting_3.csv 

In [6]:
from scipy.stats import f_oneway
averages = [all_data[x][0].loc[100]['Mean Frames'] for x in keys]
setting_1 = [averages[x] for x in range(0, len(averages), 4)]
setting_2 = [averages[x] for x in range(1, len(averages)+1, 4)]
setting_3 = [averages[x] for x in range(2, len(averages)+2, 4)]
setting_4 = [averages[x] for x in range(3, len(averages)+3, 4)]
f_score, p = f_oneway(setting_1, setting_2, setting_3, setting_4)
print(f"ANOVA F-score between settings: \nf_score = {f_score}, p={p}")

ANOVA F-score between settings: 
f_score = 0.013592550779932601, p=0.9976515171042021


In [7]:
value_1 = averages[0:4]
value_2 = averages[4:8]
value_3 = averages[8:12]
anova_score, p = f_oneway(value_1, value_2, value_3)
print(anova_score, '{0:.10f}'.format(p))

582.6425130305911 0.0000000003


# Cell_to_cell adhesion Analysis

In [8]:
c2c_names = ['c2c_adh_0_setting_1.csv',
            'c2c_adh_0_setting_2.csv',
            'c2c_adh_0_setting_3.csv',
            'c2c_adh_0_setting_4.csv',
            'c2c_adh_5_setting_1.csv',
            'c2c_adh_5_setting_2.csv',
            'c2c_adh_5_setting_3.csv',
            'c2c_adh_5_setting_4.csv',
            'c2c_adh_10_setting_1.csv',
            'c2c_adh_10_setting_2.csv',
            'c2c_adh_10_setting_3.csv',
            'c2c_adh_10_setting_4.csv',]

c2c_data = {}
for filename in c2c_names:
    c2c_data[filename] = readfile(filename)

C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(file, index_col=False)
C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(file, index_col=False)
C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(file, index_col=False)
C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(file, index_col=False)
C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: Parse

In [95]:
c2c_keys = list(c2c_data.keys())
for x in c2c_keys:
    print(f'Name is {x} with mean frames of: {c2c_data[x][0].loc[100]["Mean Frames"]}')
    print(f'Name is {x} with stdev of: {c2c_data[x][0].loc[100]["Std Frames"]}')

Name is c2c_adh_0_setting_1.csv with mean frames of: 170.1285725865486
Name is c2c_adh_0_setting_2.csv with mean frames of: 160.20317569135614
Name is c2c_adh_0_setting_3.csv with mean frames of: 149.7375002859277
Name is c2c_adh_0_setting_4.csv with mean frames of: 162.95612044933716
Name is c2c_adh_5_setting_1.csv with mean frames of: 174.96154715239308
Name is c2c_adh_5_setting_2.csv with mean frames of: 157.1272032980543
Name is c2c_adh_5_setting_3.csv with mean frames of: 154.82641245128613
Name is c2c_adh_5_setting_4.csv with mean frames of: 168.12263560413155
Name is c2c_adh_10_setting_1.csv with mean frames of: 179.93131746002373
Name is c2c_adh_10_setting_2.csv with mean frames of: 163.1496863566004
Name is c2c_adh_10_setting_3.csv with mean frames of: 148.5871427841408
Name is c2c_adh_10_setting_4.csv with mean frames of: 168.73322780772082


In [83]:
c2c_avgs = [c2c_data[x][0].loc[100]['Mean Frames'] for x in c2c_keys]
setting_1 = [c2c_avgs[x] for x in range(0, len(c2c_avgs), 4)]
setting_2 = [c2c_avgs[x] for x in range(1, len(c2c_avgs)+1, 4)]
setting_3 = [c2c_avgs[x] for x in range(2, len(c2c_avgs)+2, 4)]
setting_4 = [c2c_avgs[x] for x in range(3, len(c2c_avgs)+3, 4)]
f_score, p = f_oneway(setting_1, setting_2, setting_3, setting_4)
print(f"ANOVA F-score between settings: \nf_score = {f_score}, p={p}")

ANOVA F-score between settings: 
f_score = 4.87277861170081, p=0.03258749265358982


In [11]:
value_1 = c2c_avgs[0:4]
value_2 = c2c_avgs[4:8]
value_3 = c2c_avgs[8:12]
f_score, p = f_oneway(value_1, value_2, value_3)
print(f"ANOVA F-score between act-values: \nf_score = {f_score}, p={p}")

ANOVA F-score between act-values: 
f_score = 2.432386945816856, p=0.14304863926137187


# Pushing Power analysis

In [12]:
power_files = [
    'scenario_1-dir_50.csv',
    'scenario_2-dir_50.csv',
    'scenario_3-dir_50.csv',
    'scenario_4-dir_50.csv',
    'scenario_1-dir_100.csv',
    'scenario_2-dir_100.csv',
    'scenario_3-dir_100.csv',
    'scenario_4-dir_100.csv',
    'scenario_1-dir_200.csv',
    'scenario_2-dir_200.csv',
    'scenario_3-dir_200.csv',
    'scenario_4-dir_200.csv'
]
power_data = {}
for x in power_files:
    power_data[x] = readfile(x)

C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(file, index_col=False)
C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(file, index_col=False)
C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(file, index_col=False)
C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(file, index_col=False)
C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: Parse

In [97]:
power_keys = list(power_data.keys())
for x in power_keys:
    print(f'Name is {x} with mean frames of: {power_data[x][0].loc[100]["Mean Frames"]}')
    print(f'Name is {x} with stdev of: {power_data[x][0].loc[100]["Std Frames"]}')

Name is scenario_1-dir_50.csv with mean frames of: 2323.5615835880335
Name is scenario_1-dir_50.csv with stdev of: 456.13762603722853
Name is scenario_2-dir_50.csv with mean frames of: 2415.328790644272
Name is scenario_2-dir_50.csv with stdev of: 404.3894344110182
Name is scenario_3-dir_50.csv with mean frames of: 2403.292940566101
Name is scenario_3-dir_50.csv with stdev of: 399.31592323303926
Name is scenario_4-dir_50.csv with mean frames of: 2345.999984393813
Name is scenario_4-dir_50.csv with stdev of: 422.2251226161578
Name is scenario_1-dir_100.csv with mean frames of: 1239.5965573770493
Name is scenario_1-dir_100.csv with stdev of: 170.1285725865486
Name is scenario_2-dir_100.csv with mean frames of: 1278.223655737705
Name is scenario_2-dir_100.csv with stdev of: 160.06436784313655
Name is scenario_3-dir_100.csv with mean frames of: 1286.92
Name is scenario_3-dir_100.csv with stdev of: 150.07670138474796
Name is scenario_4-dir_100.csv with mean frames of: 1255.424426229508
Name

In [91]:
power_avgs = [list(power_data[x][0].loc[0:100]['Mean Frames']) for x in power_keys]
print(len(power_avgs))
setting_1 = [power_avgs[x] for x in range(0, len(power_avgs), 4)]
print(setting_1[2])
setting_2 = [power_avgs[x] for x in range(1, len(power_avgs)+1, 4)]
setting_3 = [power_avgs[x] for x in range(2, len(power_avgs)+2, 4)]
setting_4 = [power_avgs[x] for x in range(3, len(power_avgs)+3, 4)]
f_score, p = f_oneway(setting_1, setting_2, setting_3, setting_4)
print(f"ANOVA F-score between settings: \nf_score = {f_score}, p={p}")

12
[958.327868852459, 977.8196721311475, 952.0983606557377, 953.7868852459017, 976.9344262295082, 967.4590163934427, 963.7540983606557, 977.1147540983607, 985.9180327868852, 996.1967213114754, 974.8360655737705, 967.8852459016393, 979.1475409836065, 985.5409836065573, 994.016393442623, 968.672131147541, 981.5409836065573, 963.8524590163935, 976.360655737705, 978.4918032786885, 988.8360655737705, 965.1147540983607, 982.360655737705, 997.0327868852459, 963.8524590163935, 978.311475409836, 947.9016393442623, 999.2295081967213, 982.3770491803278, 979.6229508196722, 992.655737704918, 975.1803278688525, 981.4098360655738, 962.6065573770492, 980.5081967213115, 989.7704918032787, 970.2622950819672, 951.655737704918, 980.8852459016393, 980.4262295081967, 964.4426229508197, 980.327868852459, 977.0327868852459, 987.0491803278688, 959.7704918032787, 968.6065573770492, 971.4754098360655, 988.0983606557377, 988.8524590163935, 969.6065573770492, 955.5901639344262, 978.2622950819672, 980.0491803278688

In [15]:
value_1 = power_avgs[0:4]
value_2 = power_avgs[4:8]
value_3 = power_avgs[8:12]
f_score, p = f_oneway(value_1, value_2, value_3)
print(f"ANOVA F-score between act-values: \nf_score = {f_score}, p={p}")

ANOVA F-score between act-values: 
f_score = 2390.0451304826074, p=5.406954212489819e-13


# Obstacle size Analysis

In [43]:
names = [
    'base_1-obst_60.csv',
    'base_1-obst_55.csv',
    'base_1-obst_50.csv',
    'base_1-obst_45.csv',
    'base_1-obst_40.csv',
    'base_1-obst_35.csv',
    'base_1-obst_30.csv',
    'base_1-obst_25.csv',
    'base_1-obst_20.csv',
    'base_1-obst_0.csv',
]

standard_data = {}

for x in names:
    standard_data[x] = readfile(x)

C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(file, index_col=False)
C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(file, index_col=False)
C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(file, index_col=False)
C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(file, index_col=False)
C:\Users\jfava\AppData\Local\Temp\ipykernel_10392\571102001.py:11: Parse

In [44]:
standard_keys = list(standard_data.keys())
mean = []
std = []
min_ = []
max_ = []
for x in standard_keys:
    mean.append(standard_data[x][0].loc[100]["Mean Frames"])
    std.append(standard_data[x][0].loc[100]["Mean Frames"])
    min_.append(standard_data[x][0].loc[100]["Min Frames"])
    max_.append(standard_data[x][0].loc[100]["Max Frames"])

    print(f'Name is {x} with mean frames of: {standard_data[x][0].loc[100]["Mean Frames"]}')
    print(f'Name is {x} with std frames of: {standard_data[x][0].loc[100]["Std Frames"]}')
    print(f'Name is {x} with min frames of: {standard_data[x][0].loc[100]["Min Frames"]}')
    print(f'Name is {x} with Q1 frames of: {standard_data[x][0].loc[100]["Q1 Frames"]}')
    print(f'Name is {x} with median frames of: {standard_data[x][0].loc[100]["Median Frames"]}')
    print(f'Name is {x} with Q2 frames of: {standard_data[x][0].loc[100]["Q3 Frames"]}')
    print(f'Name is {x} with max frames of: {standard_data[x][0].loc[100]["Max Frames"]}')

Name is base_1-obst_60.csv with mean frames of: 926.9412427287149
Name is base_1-obst_60.csv with std frames of: 91.22239994362423
Name is base_1-obst_60.csv with min frames of: 748.69
Name is base_1-obst_60.csv with Q1 frames of: 868.1825
Name is base_1-obst_60.csv with median frames of: 926.81
Name is base_1-obst_60.csv with Q2 frames of: 982.7
Name is base_1-obst_60.csv with max frames of: 1162.04
Name is base_1-obst_55.csv with mean frames of: 924.4044473823371
Name is base_1-obst_55.csv with std frames of: 101.94056458818002
Name is base_1-obst_55.csv with min frames of: 736.21
Name is base_1-obst_55.csv with Q1 frames of: 859.1925
Name is base_1-obst_55.csv with median frames of: 921.065
Name is base_1-obst_55.csv with Q2 frames of: 986.46
Name is base_1-obst_55.csv with max frames of: 1213.05
Name is base_1-obst_50.csv with mean frames of: 916.1214754098353
Name is base_1-obst_50.csv with std frames of: 104.47734100192098
Name is base_1-obst_50.csv with min frames of: 726.88
Nam

In [71]:
# names = [
#     'scenario_1-setting_1.csv',
#     'scenario_2-setting_1.csv',
#     'scenario_3-setting_1.csv',
#     'scenario_4-setting_1.csv',
#     'scenario_1-setting_2.csv',
#     'scenario_2-setting_2.csv',
#     'scenario_3-setting_2.csv',
#     'scenario_4-setting_2.csv',
#     'scenario_1-setting_3.csv',
#     'scenario_2-setting_3.csv',
#     'scenario_3-setting_3.csv',
#     'scenario_4-setting_3.csv'
# ]

# standard_data = {}

#for x in names:
#    standard_data[x] = readfile(x)
avgsetting1 = []
for x in standard_keys:
    if x[11:] == 'setting_1.csv':
        avgsetting1.append(list(standard_data[x][0].loc[0:100]['Mean Frames']))
print(avgsetting1)

[[1221.704918032787, 1249.6065573770493, 1275.8360655737704, 1227.5081967213114, 1253.9344262295083, 1257.4918032786886, 1257.1475409836066, 1227.4098360655737, 1232.8360655737704, 1254.2295081967213, 1253.2459016393443, 1278.0983606557377, 1215.0983606557377, 1239.1311475409836, 1261.0655737704917, 1229.0655737704917, 1199.344262295082, 1233.9344262295083, 1238.5245901639344, 1225.032786885246, 1272.639344262295, 1253.5901639344263, 1247.9344262295083, 1209.688524590164, 1252.8360655737704, 1218.1639344262296, 1225.0819672131147, 1269.967213114754, 1215.4918032786886, 1207.344262295082, 1256.327868852459, 1246.1147540983607, 1236.704918032787, 1221.72131147541, 1223.5737704918033, 1254.4918032786886, 1236.1475409836066, 1267.5245901639344, 1275.032786885246, 1251.0819672131147, 1242.5409836065573, 1233.704918032787, 1218.8688524590164, 1256.8524590163934, 1243.4590163934427, 1248.2295081967213, 1237.1475409836066, 1268.704918032787, 1223.6065573770493, 1228.5081967213114, 1254.4754098

In [76]:
print(avgsetting1[0])

[1221.704918032787, 1249.6065573770493, 1275.8360655737704, 1227.5081967213114, 1253.9344262295083, 1257.4918032786886, 1257.1475409836066, 1227.4098360655737, 1232.8360655737704, 1254.2295081967213, 1253.2459016393443, 1278.0983606557377, 1215.0983606557377, 1239.1311475409836, 1261.0655737704917, 1229.0655737704917, 1199.344262295082, 1233.9344262295083, 1238.5245901639344, 1225.032786885246, 1272.639344262295, 1253.5901639344263, 1247.9344262295083, 1209.688524590164, 1252.8360655737704, 1218.1639344262296, 1225.0819672131147, 1269.967213114754, 1215.4918032786886, 1207.344262295082, 1256.327868852459, 1246.1147540983607, 1236.704918032787, 1221.72131147541, 1223.5737704918033, 1254.4918032786886, 1236.1475409836066, 1267.5245901639344, 1275.032786885246, 1251.0819672131147, 1242.5409836065573, 1233.704918032787, 1218.8688524590164, 1256.8524590163934, 1243.4590163934427, 1248.2295081967213, 1237.1475409836066, 1268.704918032787, 1223.6065573770493, 1228.5081967213114, 1254.47540983

In [100]:
f_score, p = f_oneway(avgsetting1[0], avgsetting1[1], avgsetting1[2], avgsetting1[3])
print(f"ANOVA F-score between settings: \nf_score = {f_score}, p={p}")


a1 = [170, 160, 149, 162] 
a2 = [174, 157, 154, 168]
a3 = [180, 163, 148, 168]
f_score, p = f_oneway(a1, a2, a3)
print(f"ANOVA F-score between settings: \nf_score = {f_score}, p={p}")

ANOVA F-score between settings: 
f_score = 93.47086082481312, p=7.41924285811945e-46
ANOVA F-score between settings: 
f_score = 0.18634458959822528, p=0.833109956745472


In [ ]:
value_1 = power_avgs[0:4]
value_2 = power_avgs[4:8]
value_3 = power_avgs[8:12]
f_score, p = f_oneway(value_1, value_2, value_3)
print(f"ANOVA F-score between act-values: \nf_score = {f_score}, p={p}")